<a href="https://colab.research.google.com/github/EmanSalah2000/sentiment_analysis/blob/main/reviews_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d umbertogriffo/googles-trained-word2vec-model-in-python

401 - Unauthorized


In [ ]:
!unzip "/content/googles-trained-word2vec-model-in-python.zip"

Archive:  /content/googles-trained-word2vec-model-in-python.zip
replace GoogleNews-vectors-negative300.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: ى
error:  invalid response [ى]
replace GoogleNews-vectors-negative300.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace GoogleNews-vectors-negative300.bin.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
!pip install stop-words
!pip install gensim==4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **1. Imports**

In [ ]:
import os
import glob
import numpy as np
import linecache
import nltk
import random
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import re
from stop_words import get_stop_words
import gensim
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import tensorflow as tf
import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
stop_words = get_stop_words('english')
from sklearn.utils import shuffle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# **2. functions**

In [ ]:
def read_sentences(path):
  myfile = open(path, 'r')
  sentences = []
  for i in range(len(myfile.readlines())):
    x = linecache.getline(path, i).strip()
    if x == "<review_text>":
      line = i + 1
      x = linecache.getline(path, line).strip()
      while x != "</review_text>":
        if len(x) >= 1:
          sentences.append(x)
        line += 1
        x = linecache.getline(path, line).strip()
      i = line
  return sentences
#####################################################################3
def get_dataset():
  path = "/content/drive/MyDrive/sorted_data_acl"
  directories = os.listdir(path)
  training_data, testing_data, training_labels, testing_labels = [], [], [], []
  for dir in directories:
    for filename in glob.glob(path + f"/{dir}/*.review"):
      if dir == "kitchen_&_housewares":
        sentences = read_sentences(filename)
        for sentence in sentences:
          testing_data.append(sentence)
          testing_labels.append(filename.split("/")[-1].replace(".review", ""))
      else:
        sentences = read_sentences(filename)
        for sentence in sentences:
          training_data.append(sentence)
          training_labels.append(filename.split("/")[-1].replace(".review", ""))
  return training_data, testing_data, training_labels, testing_labels
######################################################################################
def preprocessing(sentence):
    lemmatizer = WordNetLemmatizer()
    for word in sentence.split(" "):
      if word in stop_words:
        sentence.replace(word, "")
    sentence = re.sub(r"[-–,.%')(&|!@#$^*?=_+<>\d]", '', sentence)
    words = ""
    for word in sentence.split():
        word = lemmatizer.lemmatize(word)
        words = words + word + " "
    sentence = words
    sentence = sentence.lower()
    return sentence
#######################################################################
def get_embedings(model, data, labels):
    w2v_embeddings_sentences = []
    new_labels = []
    for i in range(len(data)):
      w2v_embeddings_sentence = []
      for word in data[i].split(" "):
        if word in list(model.index_to_key):
          w2v_embeddings_word = model[word]
          w2v_embeddings_sentence.append(w2v_embeddings_word)       
      if len(w2v_embeddings_sentence) != 0:
        w2v_embeddings_sentences.append(w2v_embeddings_sentence)
        new_labels.append(labels[i])
    return w2v_embeddings_sentences, new_labels
#########################################################################
def padding_zero(data_train,data_valid,data_test):
  
  max_num=0
  for i in range(len(data_train)):
    data_train[i]=np.array(data_train[i])
    max_num=np.maximum(max_num,data_train[i].shape[0])
  for i in range(len(data_valid)):
    data_valid[i]=np.array(data_valid[i])
    max_num=np.maximum(max_num,data_valid[i].shape[0])
  for i in range(len(data_test)):
    data_test[i]=np.array(data_test[i])
    max_num=np.maximum(max_num,data_test[i].shape[0])

  padded_data_train=np.zeros([len(data_train),max_num,data_train[0].shape[1]])
  padded_data_valid=np.zeros([len(data_valid),max_num,data_valid[0].shape[1]])
  padded_data_test=np.zeros([len(data_test),max_num,data_test[0].shape[1]])

  for i in range(len(data_train)):
    padded_data_train[i][:len(data_train[i])]=data_train[i]
  for i in range(len(data_valid)):
    padded_data_valid[i][:len(data_valid[i])]=data_valid[i]
  for i in range(len(data_test)):
    padded_data_test[i][:len(data_test[i])]=data_test[i]
  return padded_data_train,padded_data_valid,padded_data_test,max_num
###########################################################################
def encoding_labels(label):
  new_label=np.zeros([len(label),1])
  for i in range(len(label)):
    if(label[i]=='positive'):
      new_label[i]=1
    else:
      new_label[i]=0
  return new_label
###############################################################################
def bin_code(labels):
  new_label=np.zeros([len(labels),1])
  for i in range(len(labels)):
    if(labels[i]>=0.5):
      new_label[i]=1
    else:
      new_label[i]=0
  return new_label



# **3. load model**

In [ ]:
word2vec_pretrained = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)

# **4. getting dataset**

In [ ]:
training_data, testing_data, training_labels, testing_labels = get_dataset()

# **5. preprocessing data**

In [ ]:
print(len(training_data))
print(len(training_labels))
print(len(testing_data))
print(len(testing_labels))

processed_training_data = []
for sentence in training_data:
  processed_training_data.append(preprocessing(sentence))

processed_testing_data = []
for sentence in testing_data:
  processed_testing_data.append(preprocessing(sentence))

  

print(len(processed_training_data))
print(len(training_labels))
print(len(processed_testing_data))
print(len(testing_labels))

147584
147584
32882
32882
147584
147584
32882
32882


# **6.shuffle test data**

In [ ]:
# not run yet

temp = list(zip(processed_testing_data, training_labels))
random.shuffle(temp)
processed_testing_data, training_labels = zip(*temp)
processed_testing_data, training_labels = list(processed_testing_data), list(training_labels)



# not run yet 
# processed_testing_data, training_labels = shuffle(np.array(processed_testing_data), np.array(training_labels))

# **7. split train valid  for training data and shuffle it**

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(processed_training_data[:1000], training_labels[:1000], test_size=0.33, random_state=42,shuffle=True)

# **8. getting embding for train valid test**

In [ ]:
training_embedings, training_embedings_labels = get_embedings(word2vec_pretrained, X_train, y_train)
valid_embedings, valid_embedings_labels = get_embedings(word2vec_pretrained, X_valid, y_valid)
test_embedings, test_embedings_labels = get_embedings(word2vec_pretrained, processed_testing_data[:300], testing_labels[:300])

# **9. padding zero for all the data**

In [ ]:
pad_data_train,pad_data_valid,pad_data_test,max_number=padding_zero(training_embedings,valid_embedings,test_embedings)

# **10. reshape the train valid test data**

In [ ]:
pad_data_train=np.reshape(pad_data_train,(len(pad_data_train),pad_data_train.shape[1],pad_data_train.shape[2],1))
pad_data_valid=np.reshape(pad_data_valid,(len(pad_data_valid),pad_data_valid.shape[1],pad_data_valid.shape[2],1))
pad_data_test=np.reshape(pad_data_test,(len(pad_data_test),pad_data_test.shape[1],pad_data_test.shape[2],1))
pad_data_train=np.array(pad_data_train)
pad_data_valid=np.array(pad_data_valid)
pad_data_test=np.array(pad_data_test)

In [ ]:
print(pad_data_train.shape)
print(pad_data_valid.shape)
print(pad_data_test.shape)

(654, 297, 300, 1)
(321, 297, 300, 1)
(294, 297, 300, 1)


# **11. encoding the labels and convert it to binary code**

In [ ]:
y_train_update= encoding_labels(training_embedings_labels)
y_valid_update=encoding_labels(valid_embedings_labels)
y_test_update=encoding_labels(test_embedings_labels)

y_train_update= bin_code(y_train_update)
y_valid_update=bin_code(y_valid_update)
y_test_update=bin_code(y_test_update)

print(y_train_update.shape)
print(y_valid_update.shape)
print(y_test_update.shape)

(654, 1)
(321, 1)
(294, 1)


# **12. the model**

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='sigmoid', input_shape=(max_number,300,1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='sigmoid'))
model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 295, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 147, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 145, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 72, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 70, 71, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 318080)            0

# **13. fitting the model**

In [ ]:
model.fit(pad_data_train,y_train_update , epochs=5,validation_data=(pad_data_valid,y_valid_update),verbose=1)

Epoch 1/5
21/21 [==============================] - 103s 5s/step - loss: 0.3808 - accuracy: 0.9251 - val_loss: 0.3327 - val_accuracy: 0.9315
Epoch 2/5
21/21 [==============================] - 96s 5s/step - loss: 0.3464 - accuracy: 0.9251 - val_loss: 0.3006 - val_accuracy: 0.9315
Epoch 3/5
21/21 [==============================] - 96s 5s/step - loss: 0.3131 - accuracy: 0.9251 - val_loss: 0.2754 - val_accuracy: 0.9315
Epoch 4/5
21/21 [==============================] - 96s 5s/step - loss: 0.2883 - accuracy: 0.9251 - val_loss: 0.2587 - val_accuracy: 0.9315
Epoch 5/5
21/21 [==============================] - 96s 5s/step - loss: 0.2740 - accuracy: 0.9251 - val_loss: 0.2513 - val_accuracy: 0.9315


# **14. predict the model for valdation and test**

In [ ]:
y_pred_valid=model.predict(pad_data_valid)
y_pred_valid=bin_code(y_pred_valid)

In [ ]:
y_pred_test=model.predict(pad_data_test)
y_pred_test=bin_code(y_pred_test)

In [ ]:
print(y_valid_update.shape)
print(y_test_update.shape)

print(y_pred_valid.shape)
print(y_pred_test.shape)

(321, 1)
(294, 1)
(321, 1)
(294, 1)


# **15. classification report for the valdation data**

In [ ]:
print(classification_report(y_valid_update, y_pred_valid))

              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96       299
         1.0       0.00      0.00      0.00        22

    accuracy                           0.93       321
   macro avg       0.47      0.50      0.48       321
weighted avg       0.87      0.93      0.90       321



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **16. classification report for test data**

In [ ]:
 print(classification_report(y_test_update, y_pred_test))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       294

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294

